## Addition in GPT2

### Setup

In [1]:
try:
    import google.colab
    IN_COLAB = True
    # %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
    renderer = "colab"
except:
    IN_COLAB = False
    from IPython import get_ipython
    %load_ext autoreload
    %autoreload 2
    renderer = "jupyterlab"

In [2]:
%%bash
cd ../
pip install poetry
poetry install
cd notebooks

  Using cached poetry-1.5.1-py3-none-any.whl (225 kB)
  Using cached build-0.10.0-py3-none-any.whl (17 kB)
  Using cached CacheControl-0.12.14-py2.py3-none-any.whl (21 kB)
  Using cached cleo-2.0.1-py3-none-any.whl (77 kB)
  Using cached crashtest-0.4.1-py3-none-any.whl (7.6 kB)
  Using cached dulwich-0.21.5-cp310-cp310-macosx_10_9_x86_64.whl (470 kB)
  Using cached filelock-3.12.2-py3-none-any.whl (10 kB)
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
  Using cached installer-0.7.0-py3-none-any.whl (453 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 5.3 MB/s eta 0:00:00
  Using cached keyring-23.13.1-py3-none-any.whl (37 kB)
  Using cached lockfile-0.12.2-py2.py3-none-any.whl (13 kB)
  Using cached pkginfo-1.9.6-py3-none-any.whl (30 kB)
  Using cached poetry_core-1.6.1-py3-none-any.whl (450 kB)
  Using cached poetry_plugin_export-1.4.0-py3-none-any.whl (10 kB)
  Using cached pyproject_hooks-1.0.0-py3-none-any.whl (9.3 kB)
  Using cached requests-2.31.0-py3-n

Configuration file exists at /Users/theojanson/Library/Preferences/pypoetry, reusing this directory.

Consider moving TOML configuration files to /Users/theojanson/Library/Application Support/pypoetry, as support for the legacy directory will be removed in an upcoming release.


Installing dependencies from lock file

Package operations: 116 installs, 35 updates, 0 removals

  • Installing pyrsistent (0.19.3)
  • Updating six (1.16.0 /home/conda/feedstock_root/build_artifacts/six_1620240208055/work -> 1.16.0)
  • Updating traitlets (5.9.0 /home/conda/feedstock_root/build_artifacts/traitlets_1675110562325/work -> 5.9.0)
  • Installing entrypoints (0.4)
  • Installing fastjsonschema (2.17.1)
  • Updating jsonschema (4.19.0 -> 4.17.3)
  • Updating jupyter-core (5.3.1 /Users/runner/miniforge3/conda-bld/jupyter_core_1686775719487/work -> 4.12.0)
  • Updating nest-asyncio (1.5.6 /home/conda/feedstock_root/build_artifacts/nest-asyncio_1664684991461/work -> 1.5.6)
  • Updating python-dateutil (2.8.2 /home/conda/feedstock_root/build_artifacts/python-dateutil_1626286286081/work -> 2.8.2)
  • Updating pyzmq (25.1.1 /Users/runner/miniforge3/conda-bld/pyzmq_1691667536098/work -> 25.1.0)
  • Updating tornado (6.3.2 /Users/runner/miniforge3/conda-bld/tornado_1684150243998/wo